In [1]:
import bap

## Basic Disassembly

In [7]:
print '\n'.join(insn.asm for insn in bap.disasm("\x48\x83\xec\x08"))

	decl	%eax
	subl	$0x8, %esp


In [4]:
img = bap.image('arm-binaries/coreutils/coreutils_O0_ls')

In [8]:
sym = img.get_symbol('main')

In [9]:
print '\n'.join(insn.asm for insn in bap.disasm(sym))

	push	{r11, lr}
	add	r11, sp, #0x4
	sub	sp, sp, #0xc8
	str	r0, [r11, #-192]
	str	r1, [r11, #-196]
	ldr	r3, [r11, #-196]
	ldr	r3, [r3]
	mov	r0, r3
	bl	#0x10464
	mov	r0, #0x6
	ldr	r1, [pc, #2220]
	bl	#-0xecc
	ldr	r0, [pc, #2216]
	ldr	r1, [pc, #2216]
	bl	#-0xe3c
	ldr	r0, [pc, #2204]
	bl	#-0x115c
	mov	r0, #0x2
	bl	#-0xc78
	ldr	r0, [pc, #2196]
	bl	#0x19ca8
	ldr	r3, [pc, #2192]
	mov	r2, #0x0
	str	r2, [r3]
	ldr	r3, [pc, #2184]
	mov	r2, #0x1
	strb	r2, [r3]
	ldr	r3, [pc, #2176]
	mov	r2, #0x0
	str	r2, [r3]
	ldr	r3, [pc, #2168]
	mov	r2, #0x80000000
	str	r2, [r3]
	ldr	r3, [pc, #2156]
	mvn	r2, #0x0
	str	r2, [r3, #4]
	ldr	r0, [r11, #-192]
	ldr	r1, [r11, #-196]
	bl	#0x924
	str	r0, [r11, #-8]
	ldr	r3, [pc, #2132]
	ldrb	r3, [r3]
	cmp	r3, #0x0
	beq	#0x0
	bl	#0x287c
	ldr	r3, [pc, #2112]
	ldrb	r3, [r3]
	cmp	r3, #0x0
	beq	#0x19c
	mov	r0, #0xd
	bl	#-0x380
	mov	r3, r0
	cmp	r3, #0x0
	bne	#0x44
	mov	r0, #0xe
	bl	#-0x394
	mov	r3, r0
	cmp	r3, #0x0
	beq	#0xc
	ldr	r3, [pc, #2060]
	ldrb	r3, [r3]
	cmp	r3, #0x0
	bne	

## Bil visitor

In [23]:
bils = [insn.bil for insn in bap.disasm(sym)]

In [65]:
flat_bils = [b for x in bils for b in x]

In [93]:
from bap.adt import *
class MyVisitor(Visitor):
    def __init__(self):
        self.reads_sp = []
        
    def visit_Move(self, stmt):
        print "LHS:",stmt.var
        print "RHS:",stmt.expr
        self.run(stmt.expr)
        
    def visit_Var(self, var):
        if var.name == "SP":
            self.reads_sp.append(var)

In [94]:
MyVisitor().run(flat_bils[0])

LHS: Var("base_2786", Imm(0x20))
RHS: Var("SP", Imm(0x20))


In [95]:
m = lambda x: MyVisitor().run(x)
filter(None, map(m, flat_bils))

LHS: Var("base_2786", Imm(0x20))
RHS: Var("SP", Imm(0x20))
LHS: Var("mem", Mem(0x20, 0x20))
RHS: Store(Var("mem", Mem(0x20, 0x20)), PLUS(Var("base_2786", Imm(0x20)), Int(0xfffffffc, 0x20)), Var("LR", Imm(0x20)), LittleEndian(), 0x20)
LHS: Var("mem", Mem(0x20, 0x20))
RHS: Store(Var("mem", Mem(0x20, 0x20)), PLUS(Var("base_2786", Imm(0x20)), Int(0xfffffff8, 0x20)), Var("R11", Imm(0x20)), LittleEndian(), 0x20)
LHS: Var("SP", Imm(0x20))
RHS: MINUS(Var("SP", Imm(0x20)), Int(0x8, 0x20))
LHS: Var("s_2788", Imm(0x20))
RHS: Var("SP", Imm(0x20))
LHS: Var("t_2789", Imm(0x20))
RHS: Int(0x4, 0x20)
LHS: Var("R11", Imm(0x20))
RHS: PLUS(Var("SP", Imm(0x20)), Int(0x4, 0x20))
LHS: Var("s_2791", Imm(0x20))
RHS: Var("SP", Imm(0x20))
LHS: Var("t_2792", Imm(0x20))
RHS: Int(0xc8, 0x20)
LHS: Var("SP", Imm(0x20))
RHS: MINUS(Var("SP", Imm(0x20)), Int(0xc8, 0x20))
LHS: Var("mem", Mem(0x20, 0x20))
RHS: Store(Var("mem", Mem(0x20, 0x20)), PLUS(Var("R11", Imm(0x20)), Int(0xffffff40, 0x20)), Var("R0", Imm(0x20)), Litt

[]